In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from numpy import newaxis

from google.colab import drive
drive.mount('/content/gdrive/')

with open('/content/gdrive/My Drive/train_xspeech.npy', 'rb') as f:
            train_x = np.load(f)
            y = np.load(f)

shape_=np.shape(train_x)
x_train=train_x[:,newaxis,:]
tf.reshape(x_train,(shape_[0],1, shape_[1]) )
x = x_train 

In [ ]:
class DualStudent(tf.keras.Model):

    def __init__(self):
        super(DualStudent, self).__init__()
        self.nr_of_units=768
        self.nr_of_layers=5
        self.nr_of_classes=61
        self.student_1={}
        self.student_2={}
        for student in [self.student_1]:#, self.student_2]:
            for i in range(self.nr_of_layers-1):
                student["layer"+str(i+1)]=tf.keras.layers.LSTM(units=self.nr_of_units, return_sequences=True)  
            student["layer"+str(5)]=tf.keras.layers.LSTM(units=self.nr_of_units, return_sequences=False)
            student["layer"+str(6)]=tf.keras.layers.Dense(units=self.nr_of_classes)


    def call(self,  inputs):
        x1=self.student_1["layer"+str(1)](inputs)
        #x2=self.student_2["layer"+str(1)](inputs)

        for i in range(self.nr_of_layers):
            x1=self.student_1["layer"+str(i+2)](x1)
            #x2=self.student_2["layer"+str(1+2)](x2)
        return x1 #,x2


    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

model = DualStudent()
model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(x, y, epochs=1)


In [ ]:
#Functional API

class DualStudent():

    def __init__(self, nr_of_units=768, nr_of_layers=6, nr_of_classes=61, student_version="Mono_directional",show_summary=True):
        self.nr_of_units=nr_of_units
        self.nr_of_layers=nr_of_layers
        self.nr_of_classes=nr_of_classes
        self.student_version=student_version
        self.x=None
        self.y=None
        self.get_data()
        self.show_summary=show_summary
        if self.student_version=="Mono_directional":
            self.student1=self.get_model("student1")
            self.student2=self.get_model("student2")

        elif self.student_version=="Imbalanced":
            self.student1=self.get_model("student1")
            self.student2=self.get_model("student2", lstm_version="Bi_directional")

        else:
            self.student1=self.get_model("student1" , lstm_version="Bi_directional" )
            self.student2=self.get_model("student2", lstm_version="Bi_directional" )
        


    def get_data(self):
        with open('/content/gdrive/My Drive/train_xspeech.npy', 'rb') as f:
            train_x = np.load(f)
            self.y = np.load(f)

        shape_=np.shape(train_x)
        x_train=train_x[:,newaxis,:]
        tf.reshape(x_train,(shape_[0],1, shape_[1]) )
        self.x = x_train 

    def get_model(self, name_="", lstm_version="Mono_directional"):
        inputs = tf.keras.Input(shape=np.shape(self.x)[1:])

        if lstm_version=="Bi_directional":
            x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=self.nr_of_units, return_sequences=True))(inputs) 
            for i in range(self.nr_of_layers-3):
                x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=self.nr_of_units, return_sequences=True))(x)  
            x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=self.nr_of_units, return_sequences=False))(x)  
            outputs=tf.keras.layers.Dense(units=self.nr_of_classes)(x)

        else:
            x=tf.keras.layers.LSTM(units=self.nr_of_units, return_sequences=True)(inputs) 
            for i in range(self.nr_of_layers-3):
                x=tf.keras.layers.LSTM(units=self.nr_of_units, return_sequences=True)(x)  
            x=tf.keras.layers.LSTM(units=self.nr_of_units, return_sequences=False)(x)  
            outputs=tf.keras.layers.Dense(units=self.nr_of_classes)(x)

        # In case someone rather have it as one model instead of 2.
        # change the name from x to x1 and then write the same code for x2
        # and then the following line:
        # model = tf.keras.Model(inputs=inputs, outputs=[x1,x2])

        model = tf.keras.Model(inputs=inputs, outputs=outputs, name=lstm_version+"_"+name_)
        if self.show_summary:  
            model.summary()
            print("\n\n")
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, name='SGD')
        model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        return model
    
    def train(self, x=None, y=None, nr_epochs=100):
        if x==None and y==None:
            x=self.x
            y=self.y

        self.student1.fit(x, y, epochs=nr_epochs, batch_size=100)
        self.student2.fit(x, y, epochs=nr_epochs)

models={}
for version_ in ["Mono_directional", "Imbalanced", "Bi_directional"]:
    models[version_]=DualStudent(student_version=version_)
    print("\n\n\n")

models["Mono_directional"].train(nr_epochs=5)